In [1]:
import pandas as pd
import numpy as np
import string
import bs4
import importlib
import utils.preprocessing as preprocessing
import nltk
import contractions


In [80]:
nltk.__version__
bs4.__version__
contractions

<module 'contractions' from 'C:\\Users\\utilisateur\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages\\contractions\\__init__.py'>

# Data Preprocessing

In this first part we will be Preprocessing text data to prepare them for clustering and classification. This will include the following steps:
* Noise Removal
* Tekenization & Segmentation 
* Normalization

In [94]:
df = pd.read_pickle("data/dataset_business_technology_cybersecurity.pickle")
df = pd.DataFrame(df)
df.head(5)

,title,content,topic
0,Accounting,"<p class=""mw-empty-elt"">\n</p>\n\n<p><b>Accoun...",business
1,Commerce,<p><b>Commerce</b> is the exchange of goods an...,business
2,Finance,<p><b>Finance</b> is a term for matters regard...,business
3,Industrial relations,<p><b>Industrial relations</b> or <b>employmen...,business
4,Management,<p><b>Management</b> (or <b>managing</b>) is t...,business


In [98]:
# explore the data format
content = pd.DataFrame(df["content"])
content.head(5)
content.to_csv("data/content.txt")

## Noise Removal
As we are dealing with html web pages, we will need to remove all the unrelevant tokens including: html tags, javascript and css 

In [99]:
importlib.reload(preprocessing)

df["content"] = preprocessing.remove_noise_from_df(df["content"])


In [104]:
df['content'] = df['content'].str.replace(',','')

In [105]:
content = pd.DataFrame(df["content"])
content.head(5)
content.to_csv("data/content2.txt")

In [101]:
df.head(5)

,title,content,topic
0,Accounting,\n\nAccounting or accountancy is the measureme...,business
1,Commerce,Commerce is the exchange of goods and services...,business
2,Finance,Finance is a term for matters regarding the ma...,business
3,Industrial relations,Industrial relations or employment relations i...,business
4,Management,Management or managing is the administration o...,business


In [84]:
df['content'][10]

'in trade barter derived from baretor is a system of exchange where participants in a transaction directly exchange goods or services for other goods or services without using a medium of exchange such as money economists distinguish barter from gift economies in many ways barter for example features immediate reciprocal exchange not delayed in time barter usually takes place on a bilateral basis but may be multilateral ie mediated through a trade exchange in most developed countries barter usually only exists parallel to monetary systems to a very limited extent market actors use barter as a replacement for money as the method of exchange in times of monetary crisis such as when currency becomes unstable eg hyperinflation or a deflationary spiral or simply unavailable for conducting commerce\nno ethnographic studies have shown that any present or past society has used barter without any other medium of exchange or measurement nor have anthropologists found evidence that money emerged 